# TME 3.2: Projet Detection de motifs

Nom Etudiant 1: 28602815 Philippe TAN
<br>
Nom Etudiant 2: 28714617 Lasha GOGRITCHIANI

## Recheche de pattern (motifs) en utilisant les algoritmes randomisés

Les algorithmes randomisés prendre des décisions aléatoire plutôt que déterministes.
l'algorithme s'exécute différemment à chaque fois. Ils sont couramment utilisés dans situations où aucun algorithme exact et rapide est connu. Nous allons d'abord implémenter l'algorithm random Projections.


1\.  Nous allons réutiliser les fonctions du precedent pour générer t séquences artificielles de taille n, et implanter dans chaque séquence un motif de taille k à des positions aléatoires avec v substitutions choisies aléatoirement. Nous allons faire varier le motifs dans 50% de cas.

In [1]:
import random
import numpy as np

nuc = ('A', 'C', 'G', 'T')

t=5 #nombre de sequences
v=1 #nb de positions variable dans le motif
k=3 #taille du motif
n=10 #longuer des sequence
f=0.5 #frequence de variation du motif


In [2]:
def generateRandomSequence(n, upper=True):
    """
    Génère une séquence nucléotidique aléatoire 
    entrée n : longueur de la sequence
    entrée upper : bool, si True les nucléotides seront en majuscule, False minuscule
    sortie sequence : une séquence nucléotidique aléatoire 
    """
    sequence = ""

    random.seed(None)
    for i in range (0, n) :
        sequence += random.choice(nuc)
    
    if upper :
        sequence.lower()
   
    return sequence

def modifierMotif(motif, nbpos,  upper=True):
    """
    Modifie nbpos positions d'un motif aléatoirement 
    entrée motif: motif à modifier
    entrée nbpos: nombre de positions
    entrée upper : bool, si True les nucléotides modifiés seront majuscule, False minuscule
    sortie motifM: motif modifié
    """
    motifM = list(motif)
    motifM = [nt.upper() for nt in motifM]
    
    randpos = 0
    randpositions = set()
    mutation = ""
    
    i = 0
    while (i < nbpos):
        randpos = random.randint(0, len(motifM) - 1)
        if randpos not in randpositions :
            while (mutation == motifM[randpos] or mutation == ""):
                mutation = random.choice(nuc)
            motifM[randpos] = mutation
            randpositions.add(randpos)
            mutation = ""
            i += 1
    if upper :
        motifM = [nt.lower() for nt in motifM]
    
    return "".join(motifM)

#tester modifMotif
print (modifierMotif("acg", 2))

def insertMotif(sequence, motif, position):
    return sequence[:position] + motif + sequence[position:]

def implantMotifVar(k, v, t, n, f):
    """
    Génère des séquences aléatoires et les implante des motifs variables (un motif par séquence)
    entrée k: taille du motif
    entrée v: nombre de variations
    entrée t : nombre de séquences 
    entrée n : longueur des séquences
    entrée f : frequence de variation du motif.
    sortie DNA : matrice de dimension txn avec les motifs implantés
    REMARQUE : La taille totale des séquences plus motif doit être égal à t, pensez à générer de séquence aléatoire de taille t-k pour pouvoir implanter un motif de taille k
    """
    #Création du motif de base
    motif = generateRandomSequence(k, False)
    print(motif)
    sequences = []
    
    #Génère les séquences où implanter les motifs variables
    sequences = [generateRandomSequence(n - t) for i in range(k)]

    for i in range(k) :
        #Variation du motif de v nucléotides
        motifM = motif
        if random.random() < f :
            motifM = modifierMotif(motif, v)
        #Position aléatoire dans les séquences où implanter le motif modifié
        pos = random.randint(0, t - k)
        sequences[i] = insertMotif(sequences[i], motifM, pos)
    
    return sequences

adn = implantMotifVar(k, v, t, n, f)
print (adn)

adn  = [s.upper() for s in adn]
print (adn)

aat
ACT
['CAACTAAA', 'GACTCCGG', 'TAattGAC']
['CAACTAAA', 'GACTCCGG', 'TAATTGAC']


2\. Nous allons implémenter l'algorithme ``randomProjection``. D'abord, faites la fonction `getRandomFixePositions` pour générer une projection de p à k, voir un exemple dans les slides de cours. Faire aussi la fonction `generateKey` qui extrait les caractères du motif puis génère une cle qui représente la projection.

In [3]:
def getRandomFixePositions(k, p):
    """
    Genere une projection de p vers k
    entrée p: nombre de positions choisi pour la projection 
    entrée k: nombre de positions du motif original
    sortie projection: liste de positions choisi aléatoirement
    """
    projection = []
    count = 0
    for i in range (p) :
        rand = random.randint(0, k - 1)
        while rand in projection :
            rand = random.randint(0, k - 1)
        projection.append(rand)
        count += 1
   
    return projection

lR = getRandomFixePositions(7, 4)
lR.sort()
print (lR)

def generateKey(projection, motif):
    """
    extrait les caractères du motif et génère la cle de la projection
    entrée projection : liste de positions qui represent la projection
    entrée motif : motif de taille k
    sortie cle : cle de la projection
    """
    cle = ""
    
    for i in projection :
        cle += motif[i]
    
    return cle

generateKey(lR, "01234567")

[0, 1, 3, 6]


'0136'

3\. Implémenter l'algorithme ``randomProjection``. Bonnus : Pour ameliorer la performance vous pouvez abandonner les motifs de taille k peu complexes.

4\. Avez vous trouvez le motif implanté? Rexécuter l’algorithme plusieurs fois pour augmenter les chances de le trouver. 

In [9]:
def isLowComplexe(motif, m) :
    """
    Vérifie si le motif doit être éliminé en raison de sa simplicité,
    c'est-à-dire s'il a au moins m fois la même lettre ou s'il a deux lettres
    consécutives répétées.
    entrée : motif
    sortie : True si le motif doit être éliminé, False sinon
    """   
    
    if len(set(motif)) == 1 : #and len(motif) >= m: # si le motif contient une lettre m fois
        return True

    if len(set(motif)) >= 2 and len(motif) >= m :
        nuc = {'A':0, 'C':0, 'T':0, 'G':0}
        pairs = set()
        impairs = set()
        current = ""
        cpt = 0
        
        for i, letter in enumerate(motif) : #enumerate pour récupérer le nucléotide et son index
            nuc[letter] += 1
            
            if i%2 == 0 : #si lettre en position paire, on ajoute dans l'ensemble "pairs"
                pairs.add(motif[i])
            else : #sinon "impairs"
                impairs.add(motif[i])
                
        nbvalues = nuc.values()
        for i in nbvalues : #si on a au moins m occurrence d'un des nucléotides, alors le motif est SIMPLE
            if i >= m :
                return True
        if len(pairs) == 1 and len(impairs) == 1 : #si on a un pattern de deux lettres qui se répètent : SIMPLE
            return True
    
    return False

def randomProjection(k, p, sequences):
    """
    Implémente l'algorithme randomProjection
    entrée k : taille du motif
    entrée p : nombre de positions de la projection 
    entrée sequences : matrice de dimension txn contenant les séquences 
    sortie motifs : dictionaire, cle = projection, valeur= frequence
    sortie motifsSeq:  dictionaire, cle = projection, valeur= original motif
    """
    n = len(sequences[0])
    motifs  = {}; motifsSeq  = {}
    projections = getRandomFixePositions(k, p)
    projections.sort()
    
    #threshold relatif à la taille du motif
    #threshold = p // 2
    #mais ici on le fixe à 3 parce qu'on travaille sur une projection de 4 nucléotides seulement
    threshold = 3
    
    for seq in sequences:
        for i in range(n - k + 1) :
            seqisol = seq[i : i + k]
            cle = generateKey(projections, seqisol)
            if not isLowComplexe(cle, threshold) :
                if cle in motifs :
                    motifs[cle] += 1
                else :
                    motifs[cle] = 1
                
                if cle in motifsSeq :
                    motifsSeq[cle].append(seqisol)
                else :
                    motifsSeq[cle] = [seqisol]
                    
    
    return motifs, motifsSeq

#motifsSort = sorted(motifs, reverse=True, key=motifs.get)

adnTest = ['TTAACGGCAC', 'GCTCACGCCA', 'TACCGGCCGT']
motifsProj, motifsSeq = randomProjection(7, 4, adnTest)
print (motifsProj)
print (motifsSeq)

#motifsProj => {'TACG': 1, 'TCGC': 3, 'AGGA': 1, 'AGCC': 1, 'GCAG': 1, 'CACC': 1, 'CGCA': 1, 'AGGC': 1, 'CGCG': 1, 'CCCT': 1}
#motifsSeq => {'TACG': ['TTAACGG'], 'TCGC': ['TAACGGC', 'TCACGCC', 'TACCGGC'], 'AGGA': ['AACGGCA'], 'AGCC': ['ACGGCAC'], 'GCAG': ['GCTCACG'], 'CACC': ['CTCACGC'], 'CGCA': ['CACGCCA'], 'AGGC': ['ACCGGCC'], 'CGCG': ['CCGGCCG'], 'CCCT': ['CGGCCGT']}

{'TTAC': 1, 'TAAG': 1, 'AACG': 1, 'ACGC': 1, 'GCTA': 1, 'TCAG': 1, 'TACG': 1, 'ACCG': 1, 'CGGC': 1}
{'TTAC': ['TTAACGG'], 'TAAG': ['TAACGGC'], 'AACG': ['AACGGCA'], 'ACGC': ['ACGGCAC'], 'GCTA': ['GCTCACG'], 'TCAG': ['TCACGCC'], 'TACG': ['TACCGGC'], 'ACCG': ['ACCGGCC'], 'CGGC': ['CGGCCGT']}


reponse: 

5\. Implémenter la version itérative de l’algorithme ``randomProjection``. 

In [10]:

#Construire matrice de fréquence
def profile(motifs, k, nuc):
    """
    Construire une matrice de fréquence de dimension |nuc|x k
    entrée motifs: liste de motifs
    entrée k: taille du motif
    entrée nuc: alphabet
    sortie MF: matrice de fréquence
    """
    q = len(nuc)
    PWM = np.zeros((q, k))
    
    for s in motifs :
        for j in range(k) :
            if len(s) < k :
                continue
            l = s[j]
            if l == 'A':
                PWM[0][j] += 1
            if l == 'C':
                PWM[1][j] += 1
            if l == 'G':
                PWM[2][j] += 1
            if l == 'T':
                PWM[3][j] += 1
    
    return PWM

def getScore(MF, k):
    """
    Renvoie le score de MF, la somme des max de chaque colonne
    entrée MF: matrice de fréquence
    entrée k: taille du motif
    sortie sc: score
    """
    sc = 0
    
    t = len(MF)
    n = len(MF[0])
    
    for i in range(n) :
        m = 0
        for j in range(t) :
            val = MF[j][i]
            if val > m :
                m = val
        sc += m
    
    return sc

In [12]:
def randomProjIt(sequences, k, p, nuc, It):
    """
    Implémente l'algorithme randomProjection version iteractive
    entrée sequences : matrice de dimension txn contenant les séquences 
    entrée k : nombre de positions du motif
    entrée p : nombre de positions de la projection 
    entrée nuc : alphabet
    entrée It: nombre d'iterations
    sortie score : meilleur score
    sortie motifs :  liste de motifs associés au meilleur score
    """
 
    motifs = []; scores = 0
    #dico = {}
    
    for i in range(It) :
        motifsProj, motifsSeq = randomProjection(k, p, sequences)
        listmotifs = list(motifsProj.keys())
        prof = profile(listmotifs, p, nuc)
        score = getScore(prof, p)
        
        #dico[score] = listmotifs
        
        if score > scores :
            scores = score
            motifs = listmotifs
        
    return scores, motifs
#, dico

score, seqsMotif = randomProjIt(adnTest, 7, 4, nuc, 100)

print (score, seqsMotif)


20.0 ['ACGG', 'CGGC', 'GGCA', 'GCAC', 'CACG', 'ACGC', 'GCCA', 'GGCC', 'GCCG', 'CCGT']


6\. Tester l'algorithme  ``randomProjection`` sur vos données de chipSeq. N'oubliez pas de chercher les motifs dans le brin complémentaire et faire un merge de résultats. Puis générér le LOGO du motif trouvé.

In [13]:
def reverseComplement(seq):
    seq_dict = {'A':'T','T':'A','G':'C','C':'G'}
    return "".join([seq_dict[base] for base in reversed(seq)])

def complement(seq):
    seq_dict = {'A':'T','T':'A','G':'C','C':'G'}
    return "".join([seq_dict[base] for base in seq])

def printTopFMotifsFreq(motifs, m, rev=False):
    motifsRet = {}
    motifsSort = sorted(motifs, reverse=True, key=motifs.get)
    i = 0
    while (i < m):
        motifPrint = motifsSort[i]
        print (motifsSort[i])
        if rev:
            motifPrint = reverseComplement(motifsSort[i])
        print (i, motifPrint, "-", motifs[motifsSort[i]])
        i = i + 1
        
#printTopFMotifsFreq(motifs, 10)
#print (motifsSeq['TCGC'])

In [14]:
k=7; p=4; n=80

def readFasta(genome, n):
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())
    sequenceStr = "".join(sequence)
    #sequence = [sequenceStr]
    sequence = [sequenceStr[i:i+n] for i in range(0, len(sequenceStr), n)]
    sequenceRet = [x for x in sequence if x]
    return sequenceRet

genome = "Sequence_by_Peaks_4.fasta"

sequencesChip   = readFasta(genome, n)
t = len(sequencesChip)
print (sequencesChip[8], t, n, k)
revSequences = [reverseComplement(m) for m in sequencesChip]

sequences = sequencesChip + revSequences
sequencesRet = [x for x in sequencesChip if len(x) == n]

score, seqsMotif = randomProjIt(sequencesRet, k, p, nuc, 100)
print(score, seqsMotif)

ATATTAGCTCATTGTATTAGCTTTGTATTAGCTCATGATTGACATGTATATTGGCACAATTTATATTAGCTCATAAAAAA 367 80 7
192.0 ['GTAC', 'TACC', 'CCTG', 'CTGT', 'TGTA', 'GTAT', 'TATG', 'ATGC', 'TGCT', 'GCTT', 'TTGG', 'TGGC', 'GGCT', 'GCTG', 'CTGG', 'TGGT', 'TGTC', 'GTCG', 'TCGC', 'CGCA', 'GCAA', 'CAAG', 'GAGT', 'AGTA', 'ATTG', 'TTGA', 'TGAT', 'GATG', 'ATGA', 'TGCA', 'GCAG', 'CAGT', 'AGTG', 'GTTA', 'TAAT', 'AATC', 'ATCC', 'CTGA', 'TGAA', 'GAAC', 'AACC', 'ACCT', 'CCTT', 'CAGC', 'AGCT', 'GCTC', 'TCTG', 'GGCA', 'GCAC', 'ATGT', 'TACA', 'TCTA', 'CTAG', 'TAGA', 'GTAA', 'TAAC', 'AACT', 'ACTC', 'CTCG', 'TCGA', 'CGAA', 'AATT', 'ATTA', 'ATTC', 'CTAA', 'ACAG', 'ATCA', 'TCAT', 'CATA', 'GATC', 'ATAC', 'ACCG', 'CCGT', 'CGTG', 'GTGC', 'TGCC', 'CACG', 'ACGA', 'GAAT', 'TGAG', 'GAGC', 'AGCG', 'GCGT', 'CGTC', 'GTCT', 'CTTA', 'TTAA', 'AACG', 'CGAC', 'GACA', 'CAAT', 'AATG', 'TGAC', 'GACG', 'ACGC', 'GCCA', 'CCAG', 'CAGG', 'AGGC', 'GCAT', 'CATT', 'CGCT', 'CTCA', 'CAGA', 'GAAG', 'AGAT', 'GATA', 'ATAG', 'TAGC', 'AGCC', 'GCCT', 'CCTA', 'CTAT',